In [2]:
import numpy as np
import pandas as pd
import requests as req
import time
import re
import glob
import os
import subprocess
from bs4 import BeautifulSoup

# 1. Load smart contract addresses from contract-library.com

In [3]:
### Download Smart Contract Meta Data
if False:
    MAX_PAGE = 1000
    AT_PAGE = 0
    contracts = {"address": [], "block_number": [], "ether": [], "has_source": [] }
    base_url = "https://contract-library.com/api/contracts?"
    for idx in range(MAX_PAGE - AT_PAGE):
        ### Start Downloading
        page_number = idx + 1 + AT_PAGE
        end_point = "{0}n=Ethereum&q=&t=address&s=block_number&o=desc&p={1}&c=100&w=".format(base_url, page_number)
        r = req.get(end_point)
        assert(r.status_code == 200)
        print("=> page_number: {}".format(page_number))
        json = r.json()
        for contract in json["contracts"]:
            contracts["address"].append(contract["address"])
            contracts["block_number"].append(contract["block_number"])
            contracts["ether"].append(contract["ether"])
            contracts["has_source"].append(contract["has_source"])
        ## Write Fragment
        if page_number % 100 == 0:
            csv_file = "assets/addr_{}.csv".format(int(page_number / 100))
            df = pd.DataFrame(contracts)
            df.to_csv(csv_file, header=None, index=False)
            print("=> write to {}".format(csv_file))
            print("=> sleep 2 seconds")
            time.sleep(2)
            ### Clear Contract Data
            contracts["address"] = []
            contracts["block_number"] = []
            contracts["ether"] = []
            contracts["has_source"] = []

# 2. Supported version by symEvm

In [4]:
## Supported compiler versions
if False:
    versions = []
    for i in range(25 - 18 + 1):
        versions.append("0.4.{}".format(i + 18))
    for i in range(16 - 0 + 1):
        versions.append("0.5.{}".format(i))
    for i in range(3 + 1):
        versions.append("0.6.{}".format(i))
    print("=> supported version: {}".format(len(versions)))

# 3. Download source code from etherscan

In [5]:
### Filter contract source code by version
if False:
    df = pd.read_csv(
        "assets/addr.csv",
        header=None,
        names=["address", "block_number", "ether", "has_source"],
    )
    df_with_source = df[(df.has_source == True)]
    print(df_with_source.describe())
    for address in df_with_source["address"]:
        end_point = "https://etherscan.io/address/{}#code".format(address)
        r = req.get(end_point, headers={"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.113 Safari/537.36"})
        assert(r.status_code == 200)
        print("=> address: {}".format(address))
        soup = BeautifulSoup(r.text, 'html.parser')
        editor = soup.find(id="editor")
        match = re.findall(r"pragma(\s+)solidity(\s+)[^\d]*((\d+\.?)+)", editor.text)
        print(match)
        if len(match) > 0 and len(match[0]) > 2:
            version = match[0][2]
            if version in versions:
                with open("contracts/{}_{}".format(address, version), "w") as f:
                    f.write(editor.text)

# 4. Filter smart contracts by source code and version

In [1]:
### Report 1
if True:
    num_addr = !cat assets/addr.csv | wc -l
    num_addr = int(num_addr[0])
    files = !find contracts/ -type f -not -name "*.csv" -not -name "*.json"
    print("=> contracts {}".format(num_addr))
    print("=> sup_contracts {}".format(len(files)))
    print("=> % {}".format(len(files) / num_addr * 100))

=> contracts 100000
=> sup_contracts 1981
=> % 1.981


# 5. Use local compiler to filter smart contracts

In [7]:
### Filter smart contract if it can not be compiled 
if False:
    files = !find contracts/ -not -name "*.csv" -not -name "*.json"
    error_files = []
    for idx, file in enumerate(files):
        version = file.split('_')[1]
        os.environ["SYM_SOLC_VERSION"] = version
        os.environ["SYM_FILE"] = file
        !solc use "$SYM_SOLC_VERSION" > /dev/null 2>&1
        output = !solc $SYM_FILE
        if "Error" in ",".join(output):
            print("=> FAIL {}".format(file))
            error_files.append(file)
        else:
            print("=> OK {}".format(file))
    df = pd.DataFrame(error_files)
    csv_file = "contracts/errors.csv"
    df.to_csv(csv_file, header=None, index=False)

# 6. Show executable smart contracts

In [2]:
### Report 2
if True:
    files = !find contracts/ -type f -not -name "*.csv" -not -name "*.json"
    num_errors = !cat contracts/errors.csv | wc -l
    num_errors = int(num_errors[0])
    print("=> sup_contracts: {}".format(len(files)))
    print("=> error: {}".format(num_errors))
    print("=> comp : {}".format(len(files) - num_errors))
    print("=> % {}".format((len(files) - num_errors)/len(files) * 100))

=> sup_contracts: 1981
=> error: 231
=> comp : 1750
=> % 88.33922261484098


# 7. Use local compiler to compile smart contracts

In [9]:
if False:
    files = !find contracts/ -type f -not -name "*.csv" -not -name "*.json"
    df = pd.read_csv("contracts/errors.csv", header=None, names=["contract_path"])
    for contract_path in df["contract_path"]:
        if contract_path in files:
            files.remove(contract_path)
    print("=> comp: {}".format(len(files)))
    compiled_files = []
    for idx, file in enumerate(files):
        version = file.split('_')[1]
        os.environ["SYM_SOLC_VERSION"] = version
        os.environ["SYM_FILE"] = file
        os.environ["SYM_FILE_OUTPUT"] = "{}.json".format(file)
        print("=> {} / {} {}".format(idx + 1, len(files), file))
        !solc use $SYM_SOLC_VERSION > /dev/null 2>&1
        !solc --combined-json bin-runtime,srcmap-runtime,ast,asm $SYM_FILE > $SYM_FILE_OUTPUT
        compiled_files.append(file)
    df = pd.DataFrame(compiled_files)
    csv_file = "contracts/compiled.csv"
    df.to_csv(csv_file, header=None, index=False)

# 8. Show smart contract list

In [10]:
### Report 3
if True:
    df = pd.read_csv("contracts/compiled.csv", header=None, names=["contract_path"])
    num_ok = 0
    for contract_path in df["contract_path"]:
        with open(contract_path + ".json", "r") as f:
            if len(f.read()) == 0:
                continue
            num_ok = num_ok + 1
    print("=> num_ok: " + str(num_ok))

=> num_ok: 1749


# 9. Set configuration for SymEVM

In [11]:
env_content = """
dataload=02
expectCoverage=0.95
maxVisitedBlock=30
maxVisitedBlockBound=100
maxVisitedBlockStep=10
allocatedRange="a0"
""".strip()
with open(".env", "w") as f:
    f.write(env_content)

# 10. Execute smart contract by SymEVM

In [ ]:
### Run symEvm for coverage
if True:
    exceptions = {
        "address": [],
        "heap": [],
        "timeout": [],
        "others": [],
    }
    result = {
        # Concolic 
        "address": [], 
        "contract_name": [],
        "endpoints": [], 
        "covered_jumpis": [], 
        "total_jumpis": [],
        "bytelen": [],
        # Tainting analysis
        "success_sloads": [],
        "success_mloads": [],
        "success_mstores": [],
        "success_sstores": [],
        "failed_sloads": [],
        "failed_mloads": [],
        "failed_mstores": [],
        "failed_sstores": [],
        # Scanner
        "address": [],
        "integer": [],
        "disorder": [],
        "frez": [],
        "reentrancy": [],
        "delegate": [],
        "number": [],
        "timestamp": [],
        # Duration
        "concolic_duration": [],
        "tainting_duration": [],
        "scanner_duration": [],
    }
    df = pd.read_csv("contracts/compiled.csv", header=None, names=["contract_path"])
    files = ["results/sym_evm.csv", "results/exception.csv"]
    for file in files:
        if os.path.exists(file):
             os.remove(file)
        with open(file, "w") as f:
            pass
    contract_paths = df["contract_path"]
    for idx, contract_path in enumerate(contract_paths):
        os.environ["SYM_CONTRACT_FILE"] = contract_path
        os.environ["SYM_JSON_FILE"] = "{}.json".format(contract_path)
        print("{} / {} - {}".format(idx + 1, len(contract_paths), contract_path))
        ## if json file is empty
        with open(contract_path + ".json", "r") as f:
            if len(f.read()) == 0:
                continue
        ## execute SymEvm
        addr = contract_path.split('/')[2].split('_')[0]
        output = !timeout 120 node index.js
        output = '\n'.join(output)
        print(output)
        ## contract
        heap_error = "heap out of memory" in output
        has_error = "error" in output
        is_ok = !has_error
        if is_ok:
            #--------------------------------#
            #           Concolic
            #--------------------------------#
            if True:
                match = re.findall(r"Start Analyzing Contract:\s+([^\n]+)", output)
                is_ok = is_ok and len(match) > 0
                if is_ok:   
                    contract_name = match[0]
                ## endpoints
                match = re.findall(r"endpoints\s+:\s+(\d+)", output)
                is_ok = is_ok and len(match) > 0
                if is_ok:
                    endpoints = int(match[0])
                ## cjumpis
                match = re.findall(r"cjumpis\s+:\s+(\d+)", output)
                is_ok = is_ok and len(match) > 0
                if is_ok:
                    cjumpis = int(match[0])
                ## njumpis
                match = re.findall(r"njumpis\s+:\s+(\d+)", output)
                is_ok = is_ok and len(match) > 0
                if is_ok:
                    njumpis = int(match[0])
                ## byte len
                match = re.findall(r"bytelen\s+:\s+(\d+)", output)
                is_ok = is_ok and len(match) > 0
                if is_ok:
                    bytelen = int(match[0])
                ## concolic duration
                match = re.findall(r"concolic\s+:\s+(\d+)", output)
                is_ok = is_ok and len(match) > 0
                if is_ok:
                    concolic_duration = int(match[0])
            #--------------------------------#
            #           Tainting
            #--------------------------------#
            if True:
                ## success
                match = re.findall(r"success\s+:\s+\[([^\]]+)", output)
                is_ok = is_ok and len(match) > 0
                if is_ok:
                    success = list(map(lambda x: int(x), match[0].split(',')))
                ## failed
                match = re.findall(r"failed\s+:\s+\[([^\]]+)", output)
                is_ok = is_ok and len(match) > 0
                if is_ok:
                    failed = list(map(lambda x: int(x), match[0].split(',')))
                ## tainting duration
                match = re.findall(r"tainting\s+:\s+(\d+)", output)
                is_ok = is_ok and len(match) > 0
                if is_ok:
                    tainting_duration = int(match[0])
            #--------------------------------#
            #           Scanner
            #--------------------------------#
            if True:
                ## integer
                match = re.findall(r"integer\s+:\s+(false|true)", output)
                is_ok = is_ok and len(match) > 0
                if is_ok:
                    integer = 0 if match[0] == 'false' else 1
                ## disorder
                match = re.findall(r"disorder\s+:\s+(false|true)", output)
                is_ok = is_ok and len(match) > 0
                if is_ok:
                    disorder = 0 if match[0] == 'false' else 1
                ## frez
                match = re.findall(r"frez\s+:\s+(false|true)", output)
                is_ok = is_ok and len(match) > 0
                if is_ok:
                    frez = 0 if match[0] == 'false' else 1
                ## reentrancy
                match = re.findall(r"reentrancy\s+:\s+(false|true)", output)
                is_ok = is_ok and len(match) > 0
                if is_ok:
                    reentrancy = 0 if match[0] == 'false' else 1
                ## number
                match = re.findall(r"number\s+:\s+(false|true)", output)
                is_ok = is_ok and len(match) > 0
                if is_ok:
                    number = 0 if match[0] == 'false' else 1
                ## timestamp
                match = re.findall(r"timestamp\s+:\s+(false|true)", output)
                is_ok = is_ok and len(match) > 0
                if is_ok:
                    timestamp = 0 if match[0] == 'false' else 1
                ## delegate
                match = re.findall(r"delegate\s+:\s+(false|true)", output)
                is_ok = is_ok and len(match) > 0
                if is_ok:
                    delegate = 0 if match[0] == 'false' else 1
                ## tainting duration
                match = re.findall(r"scanner\s+:\s+(\d+)", output)
                is_ok = is_ok and len(match) > 0
                if is_ok:
                    scanner_duration = int(match[0])
        if is_ok:
            ## Concolic
            result["address"].append(addr)
            result["endpoints"].append(endpoints)
            result["covered_jumpis"].append(cjumpis)
            result["total_jumpis"].append(njumpis)
            result["contract_name"].append(contract_name)
            result["bytelen"].append(bytelen)
            result["concolic_duration"].append(concolic_duration)
            ## Tainting
            result["success_sloads"].append(success[0])
            result["success_mloads"].append(success[1])
            result["success_mstores"].append(success[2])
            result["success_sstores"].append(success[3])
            result["failed_sloads"].append(failed[0])
            result["failed_mloads"].append(failed[1])
            result["failed_mstores"].append(failed[2])
            result["failed_sstores"].append(failed[3])
            result["tainting_duration"].append(tainting_duration)
            ## Scanner
            result["integer"].append(integer)
            result["disorder"].append(disorder)
            result["frez"].append(frez)
            result["reentrancy"].append(reentrancy)
            result["number"].append(number)
            result["timestamp"].append(timestamp)
            result["delegate"].append(delegate)
            result["scanner_duration"].append(scanner_duration)
        else:
            exceptions["address"].append(addr)
            if heap_error:
                exceptions["heap"].append(1)
                exceptions["others"].append(0)
                exceptions["timeout"].append(0)
            elif has_error:
                exceptions["others"].append(1)
                exceptions["heap"].append(0)
                exceptions["timeout"].append(0)
            else:
                exceptions["timeout"].append(1)
                exceptions["others"].append(0)
                exceptions["heap"].append(0)
    df = pd.DataFrame(result)
    df.to_csv(files[0], index=None)
    df = pd.DataFrame(exceptions)
    df.to_csv(files[1], index=None)

1 / 1981 - contracts//0x37a4860728e292e5852b215c46dbe7a18862ef93_0.5.16
[symEvm] info: Start Analyzing Contract: BCHGateway
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 130
[symEvm] info: |	cjumpis    : 46
[symEvm] info: |	njumpis    : 46
[symEvm] info: |	bytelen    : 3278
[symEvm] info: |	concolic   : 34
[symEvm] info: |	success    : [193,1161,1049,78]
[symEvm] info: |	failed     : [0,0,0,0]
[symEvm] info: |	label      : [sloads,mloads,mstores,sstores]
[symEvm] info: |	tainting   : 1640
[symEvm] info: |	scanner    : 19402
[symEvm] info: |	integer    : true
[symEvm] info: |	disorder   : false
[symEvm] info: |	frez       : false
[symEvm] info: |	reentrancy : true
[symEvm] info: |	timestamp  : false
[symEvm] info: |	number     : false
[symEvm] info: |	delegate   : true
2 / 1981 - contracts//0x01e1bd903f14c06a5f6fd85a677c33731b3afe9e_0.4.18
[symEvm] info: Start Analyzing Contract: ConversionRates
[symEvm] info: update maxVisitedBlock to 40
[s

9 / 1981 - contracts//0xe3a0ba3f82784b37545450a0cc0dd04b08e721e1_0.5.12
[symEvm] info: Start Analyzing Contract: Date
[symEvm] info: update maxVisitedBlock to 40
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 2680
[symEvm] info: |	cjumpis    : 59
[symEvm] info: |	njumpis    : 62
[symEvm] info: |	bytelen    : 2273
[symEvm] info: |	concolic   : 733
[symEvm] info: |	success    : [0,39,5649,0]
[symEvm] info: |	failed     : [0,0,0,0]
[symEvm] info: |	label      : [sloads,mloads,mstores,sstores]
[symEvm] info: |	tainting   : 4107
[symEvm] info: |	scanner    : 198
[symEvm] info: |	integer    : false
[symEvm] info: |	disorder   : false
[symEvm] info: |	frez       : true
[symEvm] info: |	reentrancy : false
[symEvm] info: |	timestamp  : false
[symEvm] info: |	number     : false
[symEvm] info: |	delegate   : false
10 / 1981 - contracts//0x0602121722642936150f12ea1e07d62a8bb2aa71_0.5.0
[symEvm] info: Start Analyzing Contract: BOBT
[symEvm] info: -------

17 / 1981 - contracts//0x551299d87bc2ccf8ed9272fd746212a504c66fdf_0.5.0
18 / 1981 - contracts//0x5695e8dc6f08855049ac3ec5f3d0b46ffde5271a_0.5.0
[symEvm] info: Start Analyzing Contract: ProxyAdmin
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 148
[symEvm] info: |	cjumpis    : 47
[symEvm] info: |	njumpis    : 47
[symEvm] info: |	bytelen    : 2904
[symEvm] info: |	concolic   : 23
[symEvm] info: |	success    : [119,2272,1850,2]
[symEvm] info: |	failed     : [0,51,51,0]
[symEvm] info: |	label      : [sloads,mloads,mstores,sstores]
[symEvm] info: |	tainting   : 3963
[symEvm] info: |	scanner    : 4650
[symEvm] info: |	integer    : false
[symEvm] info: |	disorder   : false
[symEvm] info: |	frez       : false
[symEvm] info: |	reentrancy : false
[symEvm] info: |	timestamp  : false
[symEvm] info: |	number     : false
[symEvm] info: |	delegate   : false
19 / 1981 - contracts//0x7e4341ccd1d375433da8fd5d0cc19c51aeeb35f2_0.4.20
[symEvm] info: Start Analyz

25 / 1981 - contracts//0xd7a94d2f57948db1a9ce3b315282d3cf25c303aa_0.4.23
[symEvm] info: Start Analyzing Contract: DubaiRealEstate
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 210
[symEvm] info: |	cjumpis    : 90
[symEvm] info: |	njumpis    : 94
[symEvm] info: |	bytelen    : 8139
[symEvm] info: |	concolic   : 37
[symEvm] info: |	success    : [608,2645,1918,27]
[symEvm] info: |	failed     : [0,48,48,0]
[symEvm] info: |	label      : [sloads,mloads,mstores,sstores]
[symEvm] info: |	tainting   : 4682
[symEvm] info: |	scanner    : 1533
[symEvm] info: |	integer    : true
[symEvm] info: |	disorder   : false
[symEvm] info: |	frez       : false
[symEvm] info: |	reentrancy : false
[symEvm] info: |	timestamp  : false
[symEvm] info: |	number     : false
[symEvm] info: |	delegate   : false
26 / 1981 - contracts//0xa6cde40d861c902fb1a92d9b9c00d04dfb540e39_0.5.15
[symEvm] info: Start Analyzing Contract: ERC20Token
[symEvm] info: update maxVisitedBlock to 

35 / 1981 - contracts//0x9f623e51165102c1b7febbbbc25b76c751633155_0.5.0
[symEvm] info: Start Analyzing Contract: FixedSupplyToken
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 1227
[symEvm] info: |	cjumpis    : 64
[symEvm] info: |	njumpis    : 64
[symEvm] info: |	bytelen    : 5497
[symEvm] info: |	concolic   : 200
[symEvm] info: |	success    : [26203,29852,23622,126]
[symEvm] info: |	failed     : [0,611,611,0]
[symEvm] info: |	label      : [sloads,mloads,mstores,sstores]
[symEvm] info: |	tainting   : 48460
[symEvm] info: |	scanner    : 6094
[symEvm] info: |	integer    : false
[symEvm] info: |	disorder   : false
[symEvm] info: |	frez       : false
[symEvm] info: |	reentrancy : false
[symEvm] info: |	timestamp  : false
[symEvm] info: |	number     : false
[symEvm] info: |	delegate   : false
36 / 1981 - contracts//0x66f330546bf4c77e35a25f4a36410e507845f6b1_0.5.0
[symEvm] info: Start Analyzing Contract: TokenMintERC20Token
[symEvm] info: -------

...

938 / 1981 - contracts//0x9ced9108a16c6fb0238b0b2e475c41737a74f627_0.5.15
[symEvm] info: Start Analyzing Contract: ERC20Token
[symEvm] info: update maxVisitedBlock to 40
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 4690
[symEvm] info: |	cjumpis    : 150
[symEvm] info: |	njumpis    : 150
[symEvm] info: |	bytelen    : 13771
[symEvm] info: |	concolic   : 701
939 / 1981 - contracts//0x91ea782263c2eba2f0ee55b3f9853248e0510ddb_0.5.0
[symEvm] info: Start Analyzing Contract: TestToken
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 174
[symEvm] info: |	cjumpis    : 67
[symEvm] info: |	njumpis    : 67
[symEvm] info: |	bytelen    : 7185
[symEvm] info: |	concolic   : 33
[symEvm] info: |	success    : [113,2653,2054,20]
[symEvm] info: |	failed     : [0,48,48,0]
[symEvm] info: |	label      : [sloads,mloads,mstores,sstores]
[symEvm] info: |	tainting   : 3867
[symEvm] info: |	scanner    : 105
[symEvm] info: |	in

948 / 1981 - contracts//0x33336da3dbe1196134d56f62f5f352ef86807b76_0.6.0
[symEvm] info: Start Analyzing Contract: InstaIndex
[symEvm] info: update maxVisitedBlock to 40
[symEvm] info: update maxVisitedBlock to 50
[symEvm] info: update maxVisitedBlock to 60
[symEvm] info: update maxVisitedBlock to 70
[symEvm] info: update maxVisitedBlock to 80
[symEvm] info: update maxVisitedBlock to 90
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 150
[symEvm] info: |	cjumpis    : 84
[symEvm] info: |	njumpis    : 87
[symEvm] info: |	bytelen    : 7406
[symEvm] info: |	concolic   : 154
[symEvm] info: |	success    : [509,197,1105,100]
[symEvm] info: |	failed     : [0,2,0,0]
[symEvm] info: |	label      : [sloads,mloads,mstores,sstores]
[symEvm] info: |	tainting   : 777
[symEvm] info: |	scanner    : 3213
[symEvm] info: |	integer    : true
[symEvm] info: |	disorder   : false
[symEvm] info: |	frez       : false
[symEvm] info: |	reentrancy : true
[symEvm] info: |	t

992 / 1981 - contracts//0x50b0b1a197067878743bb33f4bdc5c814187f5df_0.5.0
[symEvm] info: Start Analyzing Contract: TokenMintERC20Token
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 1116
[symEvm] info: |	cjumpis    : 50
[symEvm] info: |	njumpis    : 50
[symEvm] info: |	bytelen    : 4809
[symEvm] info: |	concolic   : 171
993 / 1981 - contracts//0xa3c4422ec3dc5e5573b1a25792ac0bfbcd338c14_0.5.15
[symEvm] info: Start Analyzing Contract: ERC20Token
[symEvm] info: update maxVisitedBlock to 40
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 4690
[symEvm] info: |	cjumpis    : 150
[symEvm] info: |	njumpis    : 150
[symEvm] info: |	bytelen    : 13771
[symEvm] info: |	concolic   : 717
994 / 1981 - contracts//0x10dca38e1a081b080e70311b70a93effd0942802_0.5.8
^C

995 / 1981 - contracts//0x12e894578d7318b7f1ef7b013d2954b4989a3bc3_0.4.24
[symEvm] info: Start Analyzing Contract: PayingProxy
[symEvm] info: ----------

1069 / 1981 - contracts//0xaa1ecaee1b87eb5c92fe5758cc3fbb31d582e2fb_0.4.18
1070 / 1981 - contracts//0x4f71aa8f5c5c29ed8422819a4afdf64b5828af9c_0.5.15
^C

1071 / 1981 - contracts//0xcbd9f62d49ef874130e282bd35fee40f0f796741_0.5.0
1072 / 1981 - contracts//0x8159dddfb9a31d70dc1119d978395f60e9e9344f_0.5.15
^C

1073 / 1981 - contracts//0xca387101d18b9dbad0ebfded3b9698da1c004423_0.4.22
^C

1074 / 1981 - contracts//0xdb7e971d39367b20bcf4df5ae2da0fa4261bf0e8_0.5.0
^C

1075 / 1981 - contracts//0x42665308f611b022df2fd48757a457bec12ba668_0.5.0
1076 / 1981 - contracts//0xedcf2b06c6b4ca9aefd29c9d63770af7e122311e_0.5.0
^C

1077 / 1981 - contracts//0x1f1d9cd773c762ccdd8216ce77a0574105cb4c39_0.5.4
1078 / 1981 - contracts//0x7bc2505aa3c05688180a8f50200f307c160981f7_0.5.15
^C

1079 / 1981 - contracts//0x9cbc93d651115843e2bb2668b5cccf3764f60b78_0.5.0
^C

1080 / 1981 - contracts//0x4d37fce35e69c0368565ccbb53d6ea7558a964a1_0.6.0
^C

1081 / 1981 - contracts//0x00813626695bd9cbab357eedd45e5083311edeaa_0.6.0
^

1135 / 1981 - contracts//0x5d16b93c87b478fbf61d98237a0552b8c75f654e_0.5.8
[symEvm] info: Start Analyzing Contract: Token
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 1145
[symEvm] info: |	cjumpis    : 66
[symEvm] info: |	njumpis    : 66
[symEvm] info: |	bytelen    : 7367
[symEvm] info: |	concolic   : 218
[symEvm] info: |	success    : [22741,23009,19338,137]
[symEvm] info: |	failed     : [0,486,486,0]
[symEvm] info: |	label      : [sloads,mloads,mstores,sstores]
[symEvm] info: |	tainting   : 32932
assert.js:386
    throw err;
    ^

AssertionError [ERR_ASSERTION]: The expression evaluated to a falsy value:

  assert(block.name == 'Block')

    at findFunctions (/Users/wuco/Projects/symEvm/src/shared/searching.js:112:5)
    at Reentrancy.scan (/Users/wuco/Projects/symEvm/src/vul/reentrancy/index.js:88:10)
    at Scanner.scan (/Users/wuco/Projects/symEvm/src/vul/index.js:32:26)
    at /Users/wuco/Projects/symEvm/test/index.js:70:35
    at /Us